## 目的：population annealingを試す

マルコフ連鎖MCと違う点

重みが確率分布を作っていく
（重い→）

- データ
```python
'training_markers.txt','training_yields.txt'
```

- 予測アルゴリズム

χ2分類    "説明変数がカテゴリーで目的変数もカテゴリーのときは、χ2乗検定の独立性検定を用いる。"

- 最高温度

2500

始めっからK個全部入れるのは全解探索と変わらない。

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, chi2
import pprint
import itertools

In [2]:
#valiable
num_select = 3

MCS = 1

#降順である{1/b}の作成
Max_temper = 2500 #最高温度（スタート地点）
b = [1/j for j in reversed(range(1, Max_temper+1))]
print("1/bのトップバッター(最大値希望)1/b[0]は{}である".format(1/b[0]))
print("1/bのラスト1/b[-1]（最小値希望）は{}である".format(1/b[-1]))

1/bのトップバッター(最大値希望)1/b[0]は2500.0である
1/bのラスト1/b[-1]（最小値希望）は1.0である


In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）
print('the following is a part of X')
#pprint.pprint(X.head())
print("X.shape is {}".format(X.shape))
print('type of X element is {}'.format(type(X[0][0])))

data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
print('type of data_yields is {}'.format(type(data_yields)))
print('type of ori_y is {}'.format(type(ori_y)))

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

#Kをはじめにlist upするyの数とする

the following is a part of X
X.shape is (20, 20)
type of X element is <class 'numpy.int64'>
type of data_yields is <class 'pandas.core.frame.DataFrame'>
type of ori_y is <class 'pandas.core.series.Series'>


In [5]:
#状態y(yの色んなパターン)を作る
y_element = range(2) #0, 1なので
y_tuple_list = list(itertools.product(y_element, repeat=num_sample))

print("type of y_tuple_list's element is {}".format(type(y_tuple_list[0])))
print(type(pd.Series(y_tuple_list[0]))) #ori_yとtypeを揃えるためこのようにしてseriesにしないといけない
len(y_tuple_list)

#はじめに用意するレプリカはK個(今回はlen(y_tuple_list)/Max_temperとしてみる。)
#すなわちもし一回もかぶらなかったら全通り制覇できる
#K = len(y_tuple_list)/Max_temper #419.4304
K = 420

type of y_tuple_list's element is <class 'tuple'>
<class 'pandas.core.series.Series'>


In [16]:
#mask_oriの計算（以降のエネルギー計算の基準となる）
#ori_y == yとなる場合だけ、E(y)==0となる仕様

selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
selector.fit(X, ori_y)
mask_ori = selector.get_support() #各特徴量を選択したか否かのmaskを取得

In [17]:
#エネルギーが0(最低)となるのがいつか基準を先に計算しておきましょう

def E(y): #毎回のmaskとyが引数、合致度合いの計算を行う
    selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

👇同じ温度条件での比較なので、エネルギーの大きさで判断できる

だけど最後でWは計算に使いそうだから一応計算しておく。

In [18]:
"""
def cal_W(i, k):
    return　W[i-1][k]*math.exp(-(b[i]-b[i-1])*E(pd.Series(y_tuple_list[k]))) #Wは一個前のも記憶しておかないといけない
""" 

'\ndef cal_W(i, k):\n    return\u3000W[i-1][k]*math.exp(-(b[i]-b[i-1])*E(pd.Series(y_tuple_list[k]))) #Wは一個前のも記憶しておかないといけない\n'

In [8]:
def sumW(i):
    sumW[i] = 0
    for k in range(K):
        sumW[i] += W[i][k]
    return sumW[i]

👇同じ温度条件での比較なので、Wの大小でpの大小が判断できる。すなわちエネルギーの大きさでpの大小を判断できる

In [9]:
def P(i, k):
    P[i][k]=W[i][k]/sumW[i]

👇どうせ残るのはW(p)が大きいものだけ。iが増えて次のサイクルに入っても同じ。常に数をKに揃える必要があるのだろうか。

In [10]:
#i=0のときはイレギュラーなので別で作業して以降に備える

Ene = [] #これからi(温度)ごとに入れていく。一行に入っているのもlistにしたい
Ene.append({})
for k in range(K):
    Ene[-1][k]=1

high_P_Klist = []
high_P_Klist.append([ll for ll in range(K)]) #i=0のときはすべて残っている

In [11]:
for i in range(1, Max_temper):
    Ene.append({}) #iごとに行を足す
    for k in range(K):
        if k in high_P_Klist[-1]:
            Ene[-1][k] = E(pd.Series(y_tuple_list[k]))
    
    #Eneのi行目に数値が全部入ったら
    Ei_sorted = sorted(Ene[-1].items(), key=lambda x:x[1])
    tuple_kE_list = Ei_sorted[0:int(len(Ei_sorted)/2)] #上位の方々のメンバー情報が入ったtuple(k, Ene)が出来上がる
    
    #上位の方々のメンバー所属(k)が入ったlistを作る
    high_P_Klist.append([])
    for k in range(len(tuple_kE_list)):
        high_P_Klist[-1].append(tuple_kE_list[k][0])

KeyboardInterrupt: 

In [ ]:
"""
#i=0のときはイレギュラーなので別で作業して以降に備える

W = [] #これからi(温度)ごとに入れていく。一行に入っているのもlistにしたい
W.append({})
for k in range(K):
    W[-1][k]=1

high_P_Klist = []
high_P_Klist.append([ll for ll in range(K)]) #i=0のときはすべて残っている
"""

In [ ]:
"""
for i in range(1, Max_temper):
    W.append({}) #iごとに行を足す
    for k in range(K):
        if k in high_P_Klist[-1]:
            W[-1][k] = cal_W(i, k)
    
    #Wのi行目に数値が全部入ったら
    Wi_sorted = sorted(W[-1].items(), key=lambda x:x[1], reverse=True)
    tuple_kW_list = Wi_sorted[0:int(len(Wi_sorted)/2)] #上位の方々のメンバー情報が入ったtuple(k, W)が出来上がる
    
    #上位の方々のメンバー所属(k)が入ったlistを作る
    high_P_Klist.append([])
    for k in range(len(tuple_kW_list)):
        high_P_Klist[-1].append(tuple_kW_list[k][0])
"""

最後まで温度を下げて、それでも残ったkがおそらくエネルギー0のものとなっているはず。（多少前後はあるだろうけど）
残るkの個数を調べて、それがKに対してどれくらいの割合なのか知れば、いかにori_yが貴重かがわかるだろう

In [ ]:
print("温度を下げきってもなお残ったyのサンプル数は{}個である".format(len(high_P_Klist[-1])))